<a href="https://colab.research.google.com/github/thapawan/Riverine-Geomorphology-Metrics-GEE/blob/main/Water_Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# This script is specifically designed to download a single, combined water mask
# for later analysis. The mask is created by combining the outputs of multiple
# water extraction indices to improve accuracy.

import ee
import geemap
import pandas as pd
from google.colab import files, drive
import numpy as np
import datetime
import time
import os
import requests
import json
import concurrent.futures

def initialize_ee():
    """
    Authenticates and initializes the Earth Engine API.
    """
    try:
        ee.Initialize(project='skilful-boulder-440618-e7')
    except ee.EEException:
        ee.Authenticate()
        ee.Initialize(project='skilful-boulder-440618-e7')
    print("Earth Engine initialized successfully.")

def mask_clouds_s2(image):
    """
    Masks clouds, shadows, and snow from a Sentinel-2 SR image using QA60 band.
    This is analogous to the Fmask algorithm for cloud and shadow detection.
    """
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    cloud_fraction = image.get('CLOUDY_PIXEL_PERCENTAGE')
    return image.updateMask(mask).set('cloud_fraction', cloud_fraction)

def mask_clouds_landsat(image):
    """
    Masks clouds and cloud shadows from a Landsat image using QA_PIXEL band.
    This is analogous to the Fmask algorithm for cloud and shadow detection.
    """
    qa = image.select('QA_PIXEL')
    cloud_bit_mask = 1 << 3
    cloud_shadow_bit_mask = 1 << 4
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))

    cloud_fraction = ee.Image(0) # Placeholder to avoid errors

    return image.updateMask(mask).set('cloud_fraction', cloud_fraction)

def get_harmonized_image(geom, start_date, end_date, selection_method='median'):
    """
    Returns a harmonized median image (Sentinel-2 or Landsat) for a given date range.
    Filters out clouds and shadows using the methods above.
    """
    common_crs = 'EPSG:4326'

    s2_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(geom).filterDate(start_date, end_date) \
        .map(mask_clouds_s2) \
        .map(lambda img: img.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']).divide(10000).rename(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']))

    landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(geom).filterDate(start_date, end_date) \
        .map(mask_clouds_landsat) \
        .map(lambda img: img.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(0.0000275).add(-0.2).clamp(0, 1) \
                          .rename(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']))

    landsat7_collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") \
        .filterBounds(geom).filterDate(start_date, end_date) \
        .map(mask_clouds_landsat) \
        .map(lambda img: img.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).multiply(0.0000275).add(-0.2).clamp(0, 1) \
                          .rename(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']))

    landsat5_collection = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2") \
        .filterBounds(geom).filterDate(start_date, end_date) \
        .map(mask_clouds_landsat) \
        .map(lambda img: img.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).multiply(0.0000275).add(-0.2).clamp(0, 1) \
                          .rename(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']))

    combined_collection = s2_collection.merge(landsat8_collection).merge(landsat7_collection).merge(landsat5_collection)
    collection_size = combined_collection.size().getInfo()

    if collection_size > 0:
        if selection_method == 'least_cloudy':
            # Sort by cloud fraction and take the first one
            image = combined_collection.sort('cloud_fraction').first()
            if image is None: # Fallback to median if 'first' returns None
                image = combined_collection.median()
        else: # Default to median
            image = combined_collection.median()

        return image.reproject(crs=common_crs, scale=30), collection_size
    else:
        return None, collection_size

def calculate_ndwi(image):
    """
    Calculates the Normalized Difference Water Index (NDWI) and creates a binary mask (1=water, 0=non-water).
    Uses Green (B3) and NIR (B8) bands.
    """
    if image is None:
        return None
    try:
        band_names = image.bandNames().getInfo()
        if not all(band in band_names for band in ['B3', 'B8']):
            print("Missing required bands for NDWI calculation.")
            return None
        return image.normalizedDifference(['B3', 'B8']).gt(0).rename('NDWI')
    except Exception as e:
        print(f"Error calculating NDWI: {e}")
        return None

def calculate_mndwi(image):
    """
    Calculates the Modified Normalized Difference Water Index (MNDWI) and creates a binary mask.
    Uses Green (B3) and SWIR1 (B11) bands.
    """
    if image is None:
        return None
    try:
        band_names = image.bandNames().getInfo()
        if not all(band in band_names for band in ['B3', 'B11']):
            print("Missing required bands for MNDWI calculation.")
            return None
        return image.normalizedDifference(['B3', 'B11']).gt(0).rename('MNDWI')
    except Exception as e:
        print(f"Error calculating MNDWI: {e}")
        return None

def calculate_awei(image):
    """
    Calculates the Automated Water Extraction Index (AWEI) and creates a binary mask.
    Uses Green (B3), SWIR1 (B11), NIR (B8), and SWIR2 (B12) bands.
    """
    if image is None:
        return None
    try:
        band_names = image.bandNames().getInfo()
        if not all(band in band_names for band in ['B3', 'B11', 'B8', 'B12']):
            print("Missing required bands for AWEI calculation.")
            return None
        return image.expression(
            '4*(GREEN - SWIR1) - (0.25*NIR + 2.75*SWIR2)',
            {
                'GREEN': image.select('B3'), 'SWIR1': image.select('B11'),
                'NIR': image.select('B8'), 'SWIR2': image.select('B12')
            }
        ).gt(0).rename('AWEI')
    except Exception as e:
        print(f"Error calculating AWEI: {e}")
        return None

def combine_water_masks(image):
    """
    Combines NDWI, MNDWI, and AWEI masks into a single, more robust composite mask.
    A pixel is considered water if at least two of the three indices classify it as such.
    """
    if image is None:
        return None

    ndwi_mask = calculate_ndwi(image)
    mndwi_mask = calculate_mndwi(image)
    awei_mask = calculate_awei(image)

    if ndwi_mask is None or mndwi_mask is None or awei_mask is None:
        return None

    # Sum the binary masks. A value of 2 or 3 means at least two indices agree.
    combined_mask = ndwi_mask.add(mndwi_mask).add(awei_mask)

    # Create the final binary mask where water is 1 and non-water is 0.
    final_mask = combined_mask.gt(1).rename('Composite_Mask')

    return final_mask

def download_water_masks_as_tiffs(rivers_ee, year_range):
    """
    Downloads combined water/non-water masks as TIFF files for all rivers, years, and seasons.
    """
    print("Starting combined water mask download...")
    output_folder = "River_Water_Masks"

    # Get the union of all river geometries
    all_rivers_geom = rivers_ee.geometry()

    for year in year_range:
        # Wet Season (January to March)
        wet_start = f'{year}-01-01'
        wet_end = f'{year}-03-31'
        wet_image, size = get_harmonized_image(all_rivers_geom, wet_start, wet_end, selection_method='least_cloudy')

        if wet_image and size > 0:
            composite_mask = combine_water_masks(wet_image)
            if composite_mask is not None:
                description = f'All_Rivers_wet_{year}_composite_mask'
                file_name = f'All_Rivers_wet_{year}_composite_mask'

                task = ee.batch.Export.image.toDrive(
                    image=composite_mask,
                    description=description,
                    folder=output_folder,
                    fileNamePrefix=file_name,
                    region=all_rivers_geom.bounds(),
                    scale=30,
                    crs='EPSG:4326',
                    fileFormat='GeoTIFF'
                )
                task.start()
                print(f"Export task for {file_name} started.")
        else:
            print(f"No satellite data available for all rivers wet season in {year}. Skipping.")

        # Dry Season (July to September)
        dry_start = f'{year}-07-01'
        dry_end = f'{year}-09-30'
        dry_image, size = get_harmonized_image(all_rivers_geom, dry_start, dry_end, selection_method='least_cloudy')

        if dry_image and size > 0:
            composite_mask = combine_water_masks(dry_image)
            if composite_mask is not None:
                description = f'All_Rivers_dry_{year}_composite_mask'
                file_name = f'All_Rivers_dry_{year}_composite_mask'

                task = ee.batch.Export.image.toDrive(
                    image=composite_mask,
                    description=description,
                    folder=output_folder,
                    fileNamePrefix=file_name,
                    region=all_rivers_geom.bounds(),
                    scale=30,
                    crs='EPSG:4326',
                    fileFormat='GeoTIFF'
                )
                task.start()
                print(f"Export task for {file_name} started.")
        else:
            print(f"No satellite data available for all rivers dry season in {year}. Skipping.")

def main():
    initialize_ee()
    print("Mounting Google Drive...")
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return

    # Load the river shapefile from the Earth Engine asset
    try:
        rivers_ee = ee.FeatureCollection('projects/skilful-boulder-440618-e7/assets/Rivers')
        print("Shapefile loaded successfully from Earth Engine assets.")
    except Exception as e:
        print(f"Error loading shapefile from Earth Engine assets: {e}")
        return

    year_range = range(2000, 2026, 5)

    download_water_masks_as_tiffs(rivers_ee, year_range)

if __name__ == '__main__':
    main()


Earth Engine initialized successfully.
Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Shapefile loaded successfully from Earth Engine assets.
Starting combined water mask download...
Export task for All_Rivers_wet_2000_composite_mask started.
Export task for All_Rivers_dry_2000_composite_mask started.
Export task for All_Rivers_wet_2005_composite_mask started.
Export task for All_Rivers_dry_2005_composite_mask started.
Export task for All_Rivers_wet_2010_composite_mask started.
Export task for All_Rivers_dry_2010_composite_mask started.
Export task for All_Rivers_wet_2015_composite_mask started.
Export task for All_Rivers_dry_2015_composite_mask started.
Export task for All_Rivers_wet_2020_composite_mask started.
Export task for All_Rivers_dry_2020_composite_mask started.
Export task for All_Rivers_wet_2025_composite_mask started.
Export task for

In [4]:
# This script is specifically designed to download a single, combined water mask
# for later analysis. The mask is created by combining the outputs of multiple
# water extraction indices to improve accuracy.

import ee
import geemap
import pandas as pd
from google.colab import files, drive
import numpy as np
import datetime
import time
import os
import requests
import json
import concurrent.futures

def initialize_ee():
    """
    Authenticates and initializes the Earth Engine API.
    """
    try:
        ee.Initialize(project='skilful-boulder-440618-e7')
    except ee.EEException:
        ee.Authenticate()
        ee.Initialize(project='skilful-boulder-440618-e7')
    print("Earth Engine initialized successfully.")

def mask_clouds_s2(image):
    """
    Masks clouds, shadows, and snow from a Sentinel-2 SR image using QA60 band.
    This is analogous to the Fmask algorithm for cloud and shadow detection.
    """
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    cloud_fraction = image.get('CLOUDY_PIXEL_PERCENTAGE')
    return image.updateMask(mask).set('cloud_fraction', cloud_fraction)

def mask_clouds_landsat(image):
    """
    Masks clouds and cloud shadows from a Landsat image using QA_PIXEL band.
    This is analogous to the Fmask algorithm for cloud and shadow detection.
    """
    qa = image.select('QA_PIXEL')
    cloud_bit_mask = 1 << 3
    cloud_shadow_bit_mask = 1 << 4
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))

    cloud_fraction = ee.Image(0) # Placeholder to avoid errors

    return image.updateMask(mask).set('cloud_fraction', cloud_fraction)

def get_harmonized_image(geom, start_date, end_date, selection_method='median'):
    """
    Returns a harmonized median image (Sentinel-2 or Landsat) for a given date range.
    Filters out clouds and shadows using the methods above.
    """
    common_crs = 'EPSG:4326'

    s2_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(geom).filterDate(start_date, end_date) \
        .map(mask_clouds_s2) \
        .map(lambda img: img.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']).divide(10000).rename(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']))

    landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(geom).filterDate(start_date, end_date) \
        .map(mask_clouds_landsat) \
        .map(lambda img: img.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(0.0000275).add(-0.2).clamp(0, 1) \
                          .rename(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']))

    landsat7_collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") \
        .filterBounds(geom).filterDate(start_date, end_date) \
        .map(mask_clouds_landsat) \
        .map(lambda img: img.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).multiply(0.0000275).add(-0.2).clamp(0, 1) \
                          .rename(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']))

    landsat5_collection = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2") \
        .filterBounds(geom).filterDate(start_date, end_date) \
        .map(mask_clouds_landsat) \
        .map(lambda img: img.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).multiply(0.0000275).add(-0.2).clamp(0, 1) \
                          .rename(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']))

    combined_collection = s2_collection.merge(landsat8_collection).merge(landsat7_collection).merge(landsat5_collection)
    collection_size = combined_collection.size().getInfo()

    if collection_size > 0:
        if selection_method == 'least_cloudy':
            # Sort by cloud fraction and take the first one
            image = combined_collection.sort('cloud_fraction').first()
            if image is None: # Fallback to median if 'first' returns None
                image = combined_collection.median()
        else: # Default to median
            image = combined_collection.median()

        return image.reproject(crs=common_crs, scale=30), collection_size
    else:
        return None, collection_size

def calculate_ndwi(image):
    """
    Calculates the Normalized Difference Water Index (NDWI) and creates a binary mask (1=water, 0=non-water).
    Uses Green (B3) and NIR (B8) bands.
    """
    if image is None:
        return None
    try:
        band_names = image.bandNames().getInfo()
        if not all(band in band_names for band in ['B3', 'B8']):
            print("Missing required bands for NDWI calculation.")
            return None
        return image.normalizedDifference(['B3', 'B8']).gt(0).rename('NDWI')
    except Exception as e:
        print(f"Error calculating NDWI: {e}")
        return None

def calculate_mndwi(image):
    """
    Calculates the Modified Normalized Difference Water Index (MNDWI) and creates a binary mask.
    Uses Green (B3) and SWIR1 (B11) bands.
    """
    if image is None:
        return None
    try:
        band_names = image.bandNames().getInfo()
        if not all(band in band_names for band in ['B3', 'B11']):
            print("Missing required bands for MNDWI calculation.")
            return None
        return image.normalizedDifference(['B3', 'B11']).gt(0).rename('MNDWI')
    except Exception as e:
        print(f"Error calculating MNDWI: {e}")
        return None

def calculate_awei(image):
    """
    Calculates the Automated Water Extraction Index (AWEI) and creates a binary mask.
    Uses Green (B3), SWIR1 (B11), NIR (B8), and SWIR2 (B12) bands.
    """
    if image is None:
        return None
    try:
        band_names = image.bandNames().getInfo()
        if not all(band in band_names for band in ['B3', 'B11', 'B8', 'B12']):
            print("Missing required bands for AWEI calculation.")
            return None
        return image.expression(
            '4*(GREEN - SWIR1) - (0.25*NIR + 2.75*SWIR2)',
            {
                'GREEN': image.select('B3'), 'SWIR1': image.select('B11'),
                'NIR': image.select('B8'), 'SWIR2': image.select('B12')
            }
        ).gt(0).rename('AWEI')
    except Exception as e:
        print(f"Error calculating AWEI: {e}")
        return None

def combine_water_masks(image):
    """
    Combines NDWI, MNDWI, and AWEI masks into a single, more robust composite mask.
    A pixel is considered water if at least two of the three indices classify it as such.
    """
    if image is None:
        return None

    ndwi_mask = calculate_ndwi(image)
    mndwi_mask = calculate_mndwi(image)
    awei_mask = calculate_awei(image)

    if ndwi_mask is None or mndwi_mask is None or awei_mask is None:
        return None

    # Sum the binary masks. A value of 2 or 3 means at least two indices agree.
    combined_mask = ndwi_mask.add(mndwi_mask).add(awei_mask)

    # Create the final binary mask where water is 1 and non-water is 0.
    final_mask = combined_mask.gt(1).rename('Composite_Mask')

    return final_mask

def download_water_masks_as_tiffs(rivers_ee, year_range):
    """
    Downloads combined water/non-water masks as TIFF files for specified rivers, years, and seasons.
    """
    print("Starting water mask download...")
    output_folder = "River_Water_Masks"

    for feature in rivers_ee.getInfo()['features']:
        river_name = feature['properties']['GNIS_NAME']
        river_geom = ee.Feature(feature).geometry()

        for year in year_range:
            # Wet Season
            wet_start = f'{year}-01-01'
            wet_end = f'{year}-03-31'
            wet_image, size = get_harmonized_image(river_geom, wet_start, wet_end, selection_method='least_cloudy')
            if wet_image and size > 0:
                composite_mask = combine_water_masks(wet_image)
                if composite_mask is not None:
                    description = f'{river_name}_wet_{year}_composite_mask'
                    file_name = f'{river_name}_wet_{year}_composite_mask'

                    task = ee.batch.Export.image.toDrive(
                        image=composite_mask,
                        description=description,
                        folder=output_folder,
                        fileNamePrefix=file_name,
                        region=river_geom.bounds(),
                        scale=30,
                        crs='EPSG:4326',
                        fileFormat='GeoTIFF'
                    )
                    task.start()
                    print(f"Export task for {file_name} started.")
            else:
                print(f"No satellite data available for {river_name} wet season in {year}. Skipping.")

            # Dry Season
            dry_start = f'{year}-07-01'
            dry_end = f'{year}-09-30'
            dry_image, size = get_harmonized_image(river_geom, dry_start, dry_end, selection_method='least_cloudy')
            if dry_image and size > 0:
                composite_mask = combine_water_masks(dry_image)
                if composite_mask is not None:
                    description = f'{river_name}_dry_{year}_composite_mask'
                    file_name = f'{river_name}_dry_{year}_composite_mask'

                    task = ee.batch.Export.image.toDrive(
                        image=composite_mask,
                        description=description,
                        folder=output_folder,
                        fileNamePrefix=file_name,
                        region=river_geom.bounds(),
                        scale=30,
                        crs='EPSG:4326',
                        fileFormat='GeoTIFF'
                    )
                    task.start()
                    print(f"Export task for {file_name} started.")
            else:
                print(f"No satellite data available for {river_name} dry season in {year}. Skipping.")

def main():
    initialize_ee()
    print("Mounting Google Drive...")
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return

    # Load the river shapefile from the Earth Engine asset
    try:
        rivers_ee = ee.FeatureCollection('projects/skilful-boulder-440618-e7/assets/Rivers')
        print("Shapefile loaded successfully from Earth Engine assets.")
    except Exception as e:
        print(f"Error loading shapefile from Earth Engine assets: {e}")
        return

    year_range = range(2000, 2026, 5)

    download_water_masks_as_tiffs(rivers_ee, year_range)

if __name__ == '__main__':
    main()


Earth Engine initialized successfully.
Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Shapefile loaded successfully from Earth Engine assets.
Starting water mask download...
Export task for Cahaba River_wet_2000_composite_mask started.
Export task for Cahaba River_dry_2000_composite_mask started.
Export task for Cahaba River_wet_2005_composite_mask started.
Export task for Cahaba River_dry_2005_composite_mask started.
Export task for Cahaba River_wet_2010_composite_mask started.
Export task for Cahaba River_dry_2010_composite_mask started.
Export task for Cahaba River_wet_2015_composite_mask started.
Export task for Cahaba River_dry_2015_composite_mask started.
Export task for Cahaba River_wet_2020_composite_mask started.
Export task for Cahaba River_dry_2020_composite_mask started.
Export task for Cahaba River_wet_2025_composite_mask started.
Ex

KeyboardInterrupt: 